In [ ]:
# pip install psycopg2-binary sqlalchemy pandas

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, Date, Float
from sqlalchemy.engine import URL

In [3]:
oltp_url = URL.create(
    drivername="postgresql+psycopg2",
    username="guest_user",
    password="GuestUser123!",
    host="group1projectserver.postgres.database.azure.com",
    port=5432,
    database="postgres"
)

oltp_engine = create_engine(oltp_url)

In [4]:
staging_url = URL.create(
    drivername="postgresql+psycopg2",
    username="suehyun",
    password="215group1!",
    host="staging-area.postgres.database.azure.com",
    port=5432,
    database="postgres")
staging_engine = create_engine(staging_url)

In [5]:
olap_url = URL.create(
    drivername="postgresql+psycopg2",
    username="olap_admin",
    password="215db123!",
    host="215group1olap.postgres.database.azure.com",
    port=5432,
    database="postgres"
)

olap_engine = create_engine(olap_url)

In [6]:
TABLES = [
    "customer",
    "show",
    "artists",
    "instructor",
    "ticket",
    "lessons"
]

In [7]:
for table in TABLES:
    print(f"Extracting {table}...")

    # READ from OLTP (read-only)
    df = pd.read_sql(f"SELECT * FROM {table}", oltp_engine)

    # WRITE to staging
    df.to_sql(
        name=f"stg_{table}",
        con=staging_engine,
        if_exists="replace",   # overwrite per ETL run
        index=False,
        method="multi"
    )

    print(f"Loaded stg_{table} ({len(df)} rows)")

Extracting customer...
Loaded stg_customer (100 rows)
Extracting show...
Loaded stg_show (20 rows)
Extracting artists...
Loaded stg_artists (49 rows)
Extracting instructor...
Loaded stg_instructor (20 rows)
Extracting ticket...
Loaded stg_ticket (70 rows)
Extracting lessons...
Loaded stg_lessons (80 rows)


In [8]:
stg_customer = pd.read_sql("SELECT * FROM stg_Customer", staging_engine)
dim_concertgoer = stg_customer.copy()

# Keep only concertgoers (CustomerID 1 to 70)
dim_concertgoer = dim_concertgoer[dim_concertgoer['customerid'].between(1, 70)].copy()

# Create surrogate key
dim_concertgoer['ConcertGoerKey'] = range(1, len(dim_concertgoer)+1)

# Load into OLAP
dim_concertgoer.to_sql(
    'dim_concertgoer', olap_engine, if_exists='replace', index=False,
    dtype={
        'ConcertGoerKey': Integer(),
        'CustomerID': Integer(),
        'FName': String(100),
        'LName': String(100),
        'Email': String(150)
    }
)

print(f"{len(dim_concertgoer)} new rows inserted into dim_concertgoer.")

70 new rows inserted into dim_concertgoer.


In [9]:
stg_artist = pd.read_sql("SELECT * FROM stg_Artists", staging_engine)
dim_artist = stg_artist.copy()
dim_artist['artistkey'] = range(1, len(dim_artist)+1)

dim_artist.to_sql(
    'dim_artist', olap_engine, if_exists='replace', index=False,
    dtype={
        'artistkey': Integer(),
        'artistid': Integer(),
        'stagename': String(150),
        'genre': String(100)
    }
)

print(f"{len(dim_artist)} new rows inserted into dim_artist.")

49 new rows inserted into dim_artist.


In [10]:
stg_show = pd.read_sql("SELECT * FROM stg_Show", staging_engine)
dim_show = stg_show.copy()
dim_show['showkey'] = range(1, len(dim_show)+1)

dim_show.to_sql(
    'dim_Show', olap_engine, if_exists='replace', index=False,
    dtype={
        'showkey': Integer(),
        'showid': Integer(),
        'showname': String(200),
        'originalshowdate': Date()
    }
)

print(f"{len(dim_show)} new rows inserted into dim_show.")

20 new rows inserted into dim_show.


In [11]:
stg_show = pd.read_sql("SELECT showdate FROM stg_Show", staging_engine)

# Remove duplicates and reset index
dim_ticketdate = stg_show.drop_duplicates().reset_index(drop=True)

# Ensure showdate is in datetime and rename it to fulldate
dim_ticketdate['showdate'] = pd.to_datetime(dim_ticketdate['showdate'])
dim_ticketdate = dim_ticketdate.rename(columns={'showdate' : 'fulldate'})

# Create surrogate key and date attributes
dim_ticketdate['datekey'] = range(1, len(dim_ticketdate) + 1)

# Create date attributes
dim_ticketdate['Year'] = dim_ticketdate['fulldate'].dt.year
dim_ticketdate['Month'] = dim_ticketdate['fulldate'].dt.month
dim_ticketdate['dayofmonth'] = dim_ticketdate['fulldate'].dt.day 
dim_ticketdate['dayofweekname'] = dim_ticketdate['fulldate'].dt.day_name()
dim_ticketdate['dayofweek'] = dim_ticketdate['fulldate'].dt.dayofweek

# Write to OLAP
dim_ticketdate.to_sql(
    'dim_ticketdate', olap_engine, if_exists='replace', index=False,
    dtype={
        'datekey': Integer(),
        'fulldate': Date(),
        'Year': Integer(),
        'Month': Integer(),
        'dayofmonth': Integer(),
        'dayofweekname': String(20),
        'dayofweek': Integer()
    }
)

print(f"{len(dim_ticketdate)} new rows inserted into dim_ticketdate.")

20 new rows inserted into dim_ticketdate.


In [14]:
stg_ticket = pd.read_sql("SELECT * FROM stg_Ticket", staging_engine)
stg_show = pd.read_sql("SELECT * FROM stg_Show", staging_engine)
dim_concertgoer = pd.read_sql("SELECT * FROM dim_concertgoer", olap_engine)
dim_artist = pd.read_sql("SELECT * FROM dim_artist", olap_engine)
dim_ticketdate = pd.read_sql("SELECT * FROM dim_ticketdate", olap_engine)

# Merge with dimension tables
fact_ticketsales = stg_ticket.merge(stg_show, on='showid',how='left')
fact_ticketsales = fact_ticketsales.merge(dim_concertgoer, on='customerid', how='left')
fact_ticketsales = fact_ticketsales.rename(columns={'showdate' : 'fulldate'})
fact_ticketsales = fact_ticketsales.merge(dim_ticketdate, on='fulldate', how='left')

fact_ticketsales['ticketsalekey'] = range(1, len(fact_ticketsales)+1)

# Compute TicketSales fact attributes
fact_ticketsales['ticketquantity'] = fact_ticketsales['quantity']
fact_ticketsales['unitprice'] = fact_ticketsales['price']
fact_ticketsales['totalrevenue'] = fact_ticketsales['quantity'] * fact_ticketsales['price']

# Select only columns needed for Fact table
fact_ticketsales = fact_ticketsales[['ticketsalekey', 'datekey', 'customerid', 'showid',
                           'ticketquantity', 'unitprice', 'totalrevenue']]

# Load into OLAP
fact_ticketsales.to_sql(
    'fact_ticketsales', olap_engine, if_exists='replace', index=False,
    dtype={
        'ticketsalekey': Integer(),
        'datekey': Integer(),
        'customerid': Integer(),
        'showid': Integer(),
        'ticketquantity': Integer(),
        'unitprice': Float(),
        'totalrevenue': Float()
    }
)

print("TicketSales fact table created successfully!")

TicketSales fact table created successfully!


In [16]:
stg_instructor = pd.read_sql("SELECT * FROM stg_instructor", staging_engine)
dim_instructor = stg_instructor.copy()

# Create surrogate key
dim_instructor['instructorkey'] = range(1, len(dim_instructor)+1)

# Rename a column for consistency
dim_instructor = dim_instructor.rename(
    columns={'salary': 'annualsalary'}
)

# Load into OLAP
dim_instructor.to_sql('dim_instructor', olap_engine, if_exists='replace', index=False)

print(f"{len(dim_instructor)} new rows inserted into dim_instructor.")

20 new rows inserted into dim_instructor.


In [17]:
# dim_student

# Get the student subset from customers
stg_student = pd.read_sql("SELECT * FROM stg_customer WHERE customerid IN "
                            "(SELECT customerid FROM stg_lessons)",staging_engine)
dim_student = stg_student.copy()

# Create a surrogate key
dim_student['studentkey'] = range(1, len(dim_student)+1)

# Write to OLAP
dim_student.to_sql('dim_student', olap_engine, if_exists='replace', index=False)

print(f"{len(dim_student)} new rows inserted into dim_student.")

72 new rows inserted into dim_student.


In [6]:
stg_lessons = pd.read_sql("SELECT lessondate FROM stg_lessons", staging_engine)
stg_lessons['lessondate'] = pd.to_datetime(stg_lessons['lessondate'])

# Remove duplicates and reset index
dim_lsdate = stg_lessons.drop_duplicates().reset_index(drop=True)

# Create surrogate key and date attributes
dim_lsdate['datekey'] = range(1, len(dim_lsdate) + 1)
dim_lsdate['fulldate'] = dim_lsdate['lessondate']

# The next four lines use the 'fulldate' (which is datetime) column
dim_lsdate['Year'] = dim_lsdate['fulldate'].dt.year
dim_lsdate['Month'] = dim_lsdate['fulldate'].dt.month
dim_lsdate['dayofmonth'] = dim_lsdate['fulldate'].dt.day 
dim_lsdate['dayofweekname'] = dim_lsdate['fulldate'].dt.day_name()
dim_lsdate['dayofweek'] = dim_lsdate['fulldate'].dt.dayofweek

# Write to OLAP (append new rows only)
dim_lsdate.to_sql(
    'dim_lessondate', olap_engine, if_exists='append', index=False,
    dtype={
        'datekey': Integer(),
        'fulldate': Date(),
        'Year': Integer(),
        'Month': Integer(),
        'dayofmonth': Integer(),
        'dayofweekname': String(20),
        'dayofweek': Integer()
    }
)

print(f"{len(dim_lsdate)} new rows inserted into dim_lessondate.")

55 new rows inserted into dim_lessondate.


In [18]:
stg_lessonsales = pd.read_sql("SELECT instructorid, customerid, instrumenttype, lessonprice, lessondate FROM stg_lessons", staging_engine)
fact_lessonsales = stg_lessonsales.copy()
dim_lessondate = pd.read_sql("SELECT * FROM dim_lessondate", olap_engine)

# Rename a column for consistency
fact_lessonsales = fact_lessonsales.rename(columns={'lessonprice': 'lessonrevenue'})
fact_lessonsales = fact_lessonsales.rename(columns={'lessondate': 'fulldate'})

# Merge with dim_lessondate to get datekey
fact_lessonsales = fact_lessonsales.merge(dim_lessondate, on='fulldate', how='left')

# Get surrogate key
fact_lessonsales['lessonsalekey'] = range(1, len(fact_lessonsales) + 1)

# Select only columns needed for Fact table
fact_lessonsales = fact_lessonsales[['lessonsalekey', 'datekey', 'instructorid', 
                                     'customerid', 'instrumenttype', 'lessonrevenue']]

# Load into OLAP
fact_lessonsales.to_sql(
    'fact_lessonsales', olap_engine, if_exists='replace', index=False,
    dtype={
        'lessonsalekey': Integer(),
        'datekey': Integer(),
        'instructorid': Integer(),
        'customerid': Integer(),
        'instrumenttype': String(),
        'lessonrevenue': Float()
    }
)

print("LessonSales fact table created successfully!")

LessonSales fact table created successfully!
